## 전처리

In [1]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [2]:
data_path = 'D:/nlp/tensorflow-ml-nlp-tf2/4.TEXT_CLASSIFICATION/data_in/nsmc/'

train_data = pd.read_csv(data_path + 'ratings_train.txt', header = 0, 
                         delimiter = '\t', quoting = 3)


In [3]:
train_data.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [ ]:
review_text = re.sub("[^가-핳ㄱ-ㅎㅏ-ㅣ\\s]", "", train_data['document'][0])
print(review_text)

In [ ]:
okt = Okt()
review_text = okt.morphs(review_text, stem = True)
print(review_text)

In [ ]:
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한'])
clean_review = [token for token in review_text if not token in stop_words]
print(clean_review)

In [ ]:
clean_review = []
# token 
for token in review_text:
    if not token in stop_words:
        clean_review.append(token)
print(clean_review)

In [4]:
# 전처리과정 함수화!

def preprocessing(review, okt, remove_stopwords = False, stop_words = []):
    # 함수의 인자는 다음과 같다.
    # review : 전처리할 텍스트
    # okt : okt 객체를 반복적으로 생성하지 않고 미리 생성후 인자로 받는다.
    # remove_stopword : 불용어를 제거할지 선택 기본값은 False
    # stop_word : 불용어 사전은 사용자가 직접 입력해야함 기본값은 비어있는 리스트
    
    # 1. 한글 및 공백을 제외한 문자 모두 제거.
    review_text = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅣ\\s]", "", review)
    
    # 2. okt 객체를 활용해서 형태소 단위로 나눈다.
    word_review = okt.morphs(review_text, stem=True)
    
    if remove_stopwords:
        
        # 불용어 제거(선택적)
        word_review = [token for token in word_review if not token in stop_words]
        
   
    return word_review

In [5]:
stop_words = [ '은', '는', '이', '가', '하', '아', '것', '들','의', '있', '되', '수', '보', '주', '등', '한']
okt = Okt()
clean_train_review = []

for review in train_data['document']:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_train_review.append([])  #string이 아니면 비어있는 값 추가

In [6]:
test_data = pd.read_csv(data_path + 'ratings_test.txt', header=0, delimiter='\t', quoting=3 )

clean_test_review = []

for review in test_data['document']:
    # 비어있는 데이터에서 멈추지 않도록 string인 경우만 진행
    if type(review) == str:
        clean_test_review.append(preprocessing(review, okt, remove_stopwords = True, stop_words=stop_words))
    else:
        clean_test_review.append([])  #string이 아니면 비어있는 값 추가

In [10]:
token = Tokenizer()
token.fit_on_texts(clean_train_review)
train_sequences = token.texts_to_sequences(clean_train_review)
test_sequences = token.texts_to_sequences(clean_test_review)

word_vocab = token.word_index # 단어 사전 형태
word_vocab["<PAD>"] = 0


In [11]:
max_len = 8 # 문장 최대 길이

train_inputs = pad_sequences(train_sequences, maxlen=max_len, padding='post') # 학습 데이터를 벡터화
train_labels = np.array(train_data['label']) # 학습 데이터의 라벨

test_inputs = pad_sequences(test_sequences, maxlen=max_len, padding='post') # 테스트 데이터를 벡터화
test_labels = np.array(test_data['label']) # 테스트 데이터의 라벨

In [12]:
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab) # vocab size 추가

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(data_path):
    os.makedirs(data_path)

# 전처리 된 학습 데이터를 넘파이 형태로 저장
np.save(open(data_path + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(data_path + TRAIN_LABEL_DATA, 'wb'), train_labels)

# 전처리 된 테스트 데이터를 넘파이 형태로 저장
np.save(open(data_path + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(data_path + TEST_LABEL_DATA, 'wb'), test_labels)

# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(data_path + DATA_CONFIGS, 'w'), ensure_ascii=False)